# Tarea 2


## 1.- Introducción

### Objetivo

En esta tarea abarcaremos la resolución de problemas de [Sequence Labelling](https://en.wikipedia.org/wiki/Sequence_labeling) (etiquetado de sequencias).

**¿Qué es sequence labelling?** 

En breves palabras, dada una secuencia de tokens (frase u oración) sequence labelling tiene por objetivo asignar una etiqueta a cada token de dicha secuencia.

**Named Entity Recognition (NER)**

Esta tarea consiste en localizar y clasificar los tokens de una oración que representen entidades nombradas. Es decir, tokens que simbolicen (1) **personas**, (2) **organizaciones**, (3) **lugares** y (4) **adjetivos, eventos y otras entidades que no entren en las categorías anteriores** deberán ser taggeados como (1) **PER**, (2) **ORG**, (3) **LOC** y (4) **MISC** respectivamente. Adicionalmente, dado que existen entidades representadas en más de un token (como La Serena), se utiliza la notación BIO como prefijo al tag: Beginning, Inside, Outside. Es decir, si encuentro una entidad, el primer token etiquetado será precedido por B, el segundo por I y los n restantes por I. Por otra parte, si el token no representa ninguna entidad nombrada, se representa por O. Un ejemplo de esto es:

Por ejemplo:

```
Felipe B-PER
Bravo I-PER
es O
el O
profesor O
de O
PLN B-MISC
de O
la O
Universidad B-ORG
de I-ORG
Chile I-ORG
. O
```

Para mayor información, visitar [NER en Wikipedia](https://es.wikipedia.org/wiki/Reconocimiento_de_entidades_nombradas).

**En esta tarea usaremos un dataset de noticias etiquetadas en español.**



### Métodos

La idea principal de este notebook es tener un baseline para implementar la tarea 2. Para esto, construiremos una solución utilizando una [red recurrente](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-RNN.pdf)  `LSTM` simple (un solo nivel y sin bi-direccionalidad). 

En este notebook veremos el proceso de cargar los datasets, haremos batches de texto y padding para poder entrenar el modelo utilizando los batches.

Finalmente, mostraremos como construir un output para que lo puedan probar en la tarea en codelab.

Se espera que para la tarea, ustedes complementen y mejoren el baseline a partir de varias mejoras que pueden ser aplicadas al modelo. Algunas sugerencias:

*   Inicializar los embeddings con modelos pre-entrenados. (glove, etc...)
*   Probar bi-direccionalidad.
*   Probar varios niveles en las redes recurrentes.
*   Probar teacher forcing.
*   Incluir dropout.







## 2.- Preprocesamiento

Para el pre-procesamiento utilizaremos la biblioteca [`torchtext`](https://github.com/pytorch/text). Como su nombre lo indica, torchtext brinda funcionalidades para hacer procesamiento de texto con pytorch.

### Algunas referencias 


https://github.com/pytorch/text

https://pytorch.org/text/index.html

https://torchtext.readthedocs.io/en/latest/

In [0]:
# Instalar torchtext
!pip3 install --upgrade torchtext

     |████████████████████████████████| 61kB 2.3MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
import torch
from torchtext import data, datasets

In [0]:
#Garantizar reproducibilidad
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Torchtext: Data y Fields

 `TORCHTEXT.DATA` 

El primer módulo que usaremos, `DATA`, nos proporciona las siguientes funcionalidades:

*   Capacidad para definir un pipeline de preprocesamiento
*   Batching, padding, y numericalizing (incluida la creación de un vocabulario)
*   Divisiones de conjuntos de datos (entrenamiento, validación, prueba)
*   Cargar datasets personalizados

`DATA.Field` 

* Define un tipo de datos junto con instrucciones para convertir el texto a Tensor.
* La clase Field modela tipos de datos comunes para procesamiento de texto  que pueden ser representados por tensores.
* Contiene un objeto Vocab que define el conjunto de valores posibles para elementos del Field y sus representaciones numéricas correspondientes.
* El objeto Field también contiene otros parámetros relacionados con la forma en que se debe numericalizar un tipo de datos, como un método de tokenización y el tipo de Tensor que se debe producir.
* Si un Field se comparte entre dos columnas en un conjunto de datos (por ejemplo, preguntas y respuestas en un dataset de Question Answering), tendrán un vocabulario compartido.




### Definir nuestros fields


Recordemos como está definido el dataset:

```
El O
Abogado B-PER
General I-PER
del I-PER
Estado I-PER
, O
Daryl B-PER
Williams I-PER
```


A partir de esto, podemos definir los siguientes fields:

In [0]:
# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = data.Field(lower=False)

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = data.Field(unk_token=None)

fields = (("text", TEXT), ("nertags", NER_TAGS))

### Obtener datos

Ahora, obtenemos los datos desde la fuente:

In [0]:
%%capture
!wget https://github.com/dccuchile/CC6205/releases/download/Data/train_NER_esp.txt  # Dataset de Entrenamiento
!wget https://github.com/dccuchile/CC6205/releases/download/Data/val_NER_esp.txt    # Dataset de Validación (Para probar y ajustar el modelo)
!wget https://github.com/dccuchile/CC6205/releases/download/Data/test_NER_esp.txt   # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!

### Cargar datasets usando SequenceTaggingDataset


Ahora, usamos la clase `sequenceTaggingDataset` para cargar los datasets.

El objetivo de cargar por medio de esta clase los datasets es poder cargar los datos de forma comprensible para pytorch. Para esto, usan el arreglo `field` para definir como se leera cada linea.   



In [0]:
train_data = datasets.SequenceTaggingDataset('train_NER_esp.txt', fields, encoding="iso-8859-1", separator=" ")
valid_data = datasets.SequenceTaggingDataset('val_NER_esp.txt', fields, encoding="iso-8859-1", separator=" ")
test_data = datasets.SequenceTaggingDataset('test_NER_esp.txt', fields, encoding="iso-8859-1", separator=" ")

In [0]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 8323
Number of validation examples: 1915
Number of testing examples: 1517


Visualizemos un ejemplo

In [0]:
random_train_item = torch.randint(0, len(train_data), [1])

random_example = vars(train_data.examples[random_train_item])

for word, tag in zip(random_example['text'], random_example['nertags']):
  print(word, '\t', tag)

La 	 O
quinta 	 O
edición 	 O
del 	 O
Festival 	 B-MISC
de 	 I-MISC
Teatro 	 I-MISC
en 	 I-MISC
la 	 I-MISC
Calle 	 I-MISC
, 	 O
que 	 O
organiza 	 O
anualmente 	 O
el 	 O
Ayuntamiento 	 B-ORG
de 	 I-ORG
Villanueva 	 I-ORG
de 	 I-ORG
la 	 I-ORG
Serena 	 I-ORG
con 	 O
la 	 O
colaboración 	 O
de 	 O
la 	 O
Consejería 	 B-ORG
de 	 I-ORG
Cultura 	 I-ORG
de 	 I-ORG
la 	 I-ORG
Junta 	 I-ORG
de 	 I-ORG
Extremadura 	 I-ORG
, 	 O
tendrá 	 O
lugar 	 O
del 	 O
12 	 O
al 	 O
15 	 O
de 	 O
julio 	 O
, 	 O
anunció 	 O
hoy 	 O
el 	 O
concejal 	 O
de 	 O
Cultura 	 B-MISC
villanovense 	 O
, 	 O
Gregorio 	 B-PER
Gil 	 I-PER
Ruedas 	 I-PER
. 	 O


### Construir los vocabularios para el texto y las etiquetas

El siguiente paso consiste en construir los vocabularios: Objetos que contienen todos los tokens (de entrenamiento) posibles para ambos fields. Para esto, hacemos uso del método `Field.build_vocab` sobre cada uno de los fields. 

Si se proporciona un objeto Dataset, se utilizan todas las columnas correspondientes a este field. También se pueden proporcionar columnas individuales directamente.

In [0]:

TEXT.build_vocab(train_data) #,
                 # unk_init = torch.Tensor.normal_,
                 #vectors = "glove.6B.100d") #Esto es una pista de como usar vectores pre-entrenados.

NER_TAGS.build_vocab(train_data)

In [0]:
print(f"Tokens únicos en TEXT: {len(TEXT.vocab)}")
print(f"Tokens únicos en NER_TAGS: {len(NER_TAGS.vocab)}")

Tokens únicos en TEXT: 26101
Tokens únicos en NER_TAGS: 10


Veamos las posibles etiquetas que se le pueden asignar a una palabra:

In [0]:
print(NER_TAGS.vocab.itos)

['<pad>', 'O', 'B-ORG', 'I-ORG', 'B-LOC', 'B-PER', 'I-PER', 'I-MISC', 'B-MISC', 'I-LOC']


Las cuales coinciden con lo estipulado al comienzo. La única diferencia es la presencia de \<pad\>, el cual es generado por el dataloader.

Veamos ahora los tokens mas frecuentes y especiales:

In [0]:
# Tokens mas frecuentes
print(TEXT.vocab.freqs.most_common(20))

[('de', 17657), (',', 14716), ('la', 9571), ('que', 7516), ('.', 7263), ('el', 6905), ('en', 6484), ('"', 5691), ('y', 5336), ('a', 4304), ('del', 3742), ('los', 3705), ('se', 2495), ('por', 2473), ('las', 2368), (')', 2099), ('(', 2094), ('con', 2001), ('un', 1985), ('para', 1692)]


In [0]:
# Tokens con los primeros 10 indices del vocabulario.
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'de', ',', 'la', 'que', '.', 'el', 'en', '"']


In [0]:
#Tokens especiales en el texto.
for tok in TEXT.vocab.itos:
    if tok[0] == "<":
        print(tok)

<unk>
<pad>


### Frecuencia de los Tags

Visualizemos rápidamente las cantidades y frecuencias de cada tag:

In [0]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages

print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag Ocurrencia Porcentaje

O	231920	87.6%
B-ORG	7390	 2.8%
I-ORG	4992	 1.9%
B-LOC	4913	 1.9%
B-PER	4321	 1.6%
I-PER	3903	 1.5%
I-MISC	3212	 1.2%
B-MISC	2173	 0.8%
I-LOC	1891	 0.7%


## 3.- El modelo

### Definición de la red

Teniendo ya cargado los datos, toca definir nuestro modelo.

In [0]:
BATCH_SIZES = (64,2,2)

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dividir datos entre entrenamiento y test
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_sizes = BATCH_SIZES,
    device = device,
    sort=False)

La red base tendrá una capa de embedding, unas cuantas LSTM y una capa de salida. Además, contará con dropout para el entrenamiento.

Esta será definida en la siguiente clase:


In [0]:
import torch.nn as nn
import torch.nn.functional as F

# Definir la red
class RNNNer(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        # Capa de embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # Capa LSTM
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers = n_layers, 
                           bidirectional = bidirectional)
        
        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        # Convertir lo enviado a embedding
        embedded = F.relu(self.dropout(self.embedding(text)))
        
        #embedded = [sent len, batch size, emb dim]
        
        # Pasar los embeddings por la rnn (LSTM)
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]
        
        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        
        #predictions = [sent len, batch size, output dim]
        
        return predictions

## Inicialización de la red

Ahora, definimos los hiperparámetros de la red:

In [0]:
# Hiperparámetros de la red

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 128
HIDDEN_DIM = 512
OUTPUT_DIM = len(NER_TAGS.vocab)
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.25 
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Creamos nuestro modelo.
model = RNNNer(INPUT_DIM, 
                     EMBEDDING_DIM, 
                     HIDDEN_DIM, 
                     OUTPUT_DIM, 
                     N_LAYERS, 
                     BIDIRECTIONAL, 
                     DROPOUT, 
                     PAD_IDX)

E iniciamos los pesos de la red de forma aleatoria (Usando una distribución normal).

In [0]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1)
        
model.apply(init_weights)

RNNNer(
  (embedding): Embedding(26101, 128, padding_idx=1)
  (rnn): LSTM(128, 512, num_layers=3, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model):,} parámetros entrenables.')

El modelo tiene 18,580,106 parámetros entrenables.


In [0]:
#pretrained_embeddings = TEXT.vocab.vectors
#print(pretrained_embeddings.shape)

In [0]:
#model.embedding.weight.data.copy_(pretrained_embeddings)

Por último, definimos los embeddings que representan a  \<UNK\> y \<PAD\> como [0, 0, ..., 0]

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# Los primeros 2 embeddings representan UNK y PAD
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0931, -0.0657,  0.0369,  ...,  0.0365, -0.0111, -0.0174],
        ...,
        [-0.2111,  0.0855, -0.0953,  ..., -0.1742,  0.1881, -0.0942],
        [-0.0228, -0.1057,  0.1015,  ..., -0.0419, -0.0931,  0.1250],
        [-0.0398,  0.0244, -0.0247,  ..., -0.0619, -0.1174,  0.1016]])


## Entrenamiento y evaluación de la red

Teniendo ya definida la red, comenzamos con el proceso de entrenamiento.
Lo primero es definir el optimizador (`ADAM`) y la loss que usaremos (`CrossEntropy`).

Referencias: https://github.com/dccuchile/CC6205/blob/master/slides/NLP-linear.pdf

In [0]:
import torch.optim as optim

# Optimizador
optimizer = optim.Adam(model.parameters())

TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]

# Loss: Cross Entropy
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [0]:
# Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
model = model.to(device)
criterion = criterion.to(device)

Por otra parte, definiremos las métricas que serán usadas en la competencia: `precision`, `recall` y `f1`

In [0]:
# Definimos las métricas

from sklearn.metrics import f1_score, precision_score, recall_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)


def calculate_metrics(preds, y):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim = 1, keepdim = True) 
    # Obtenemos los indices distintos de 0.

    y_pred = y_pred.view(-1).to('cpu')
    y_true = y.to('cpu')

    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')

    return precision, recall, f1


### Definimos el entrenamiento

Algunos conceptos previos: 

- `epoca` : una pasada completa de una dataset.
- `batch`: una fracción de la época. Se utilizan para entrenar mas rápidamente la red. (mas eficiente pasar n datos que uno en cada ejecución del backpropagation)



#### `train`

Esta función está encargada de entrenar la red en una época. Para esto, por cada batch de la época actual, predice los tags del texto, calcula su loss y luego hace backpropagation para actualizar los pesos de la red."

In [0]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0
    
    model.train()
    
    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado    
        text = batch.text
        tags = batch.nertags
        
        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        # Predecimos los tags del texto del batch.
        predictions = model(text)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
                
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)
        
        # Calculamos los gradientes
        loss.backward()
        
        # Actualizamos los parámetros de la red
        optimizer.step()
        
        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

        
    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

#### `evaluate`

Evalua el rendimiento actual de la red usando los datos de validación. 

Por cada batch de estos datos, calcula y reporta el loss y las métricas asociadas al conjunto de validación. 

Ya que las métricas son calculadas por cada batch, estas son retornadas promediadas por el número de batches entregados. (ver linea del return)

In [0]:
# Evaluamos el modelo 

def evaluate(model, iterator, criterion):

    
    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0
    
    model.eval()
    
    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos 
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)
                    
            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)
            
            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Entrenamiento de la red

En este cuadro de código ejecutaremos el entrenamiento de la red.
Para esto, primero definiremos el número de épocas y luego por cada época, ejecutaremos `train` y `evaluate`.

**Importante: Reiniciar el modelo**

Si ejecutas nuevamente esta celda, se seguira entrenando el mismo modelo una y otra vez. 
Para reiniciar el modelo se debe ejecutar nuevamente la celda que contiene la función `init_weights`



In [0]:
N_EPOCHS = 15

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(model, train_iterator, optimizer, criterion)

    # Evaluar
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'modelo_tarea_2.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')

Epoch: 01 | Epoch Time: 0m 23s
	Train Loss: 0.695 | Train f1: 0.05 | Train precision: 0.04 | Train recall: 0.10
	 Val. Loss: 0.804 |  Val. f1: 0.17 | Val. precision: 0.14 | Val. recall: 0.22
Epoch: 02 | Epoch Time: 0m 23s
	Train Loss: 0.621 | Train f1: 0.05 | Train precision: 0.04 | Train recall: 0.10
	 Val. Loss: 0.766 |  Val. f1: 0.17 | Val. precision: 0.14 | Val. recall: 0.22
Epoch: 03 | Epoch Time: 0m 23s
	Train Loss: 0.507 | Train f1: 0.10 | Train precision: 0.14 | Train recall: 0.15
	 Val. Loss: 0.444 |  Val. f1: 0.31 | Val. precision: 0.31 | Val. recall: 0.35
Epoch: 04 | Epoch Time: 0m 23s
	Train Loss: 0.218 | Train f1: 0.34 | Train precision: 0.37 | Train recall: 0.39
	 Val. Loss: 0.400 |  Val. f1: 0.37 | Val. precision: 0.38 | Val. recall: 0.41
Epoch: 05 | Epoch Time: 0m 23s
	Train Loss: 0.156 | Train f1: 0.48 | Train precision: 0.51 | Train recall: 0.53
	 Val. Loss: 0.328 |  Val. f1: 0.37 | Val. precision: 0.37 | Val. recall: 0.43
Epoch: 06 | Epoch Time: 0m 23s
	Train Loss: 0

### Cargar modelo preentrenado


In [0]:
model.load_state_dict(torch.load('modelo_tarea_2.pt'))

<All keys matched successfully>

### Resultados finales del set de validación

Estos son los resultados de predecir el dataset de evaluación con el modelo actualmente entrenado:

In [0]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')

Val. Loss: 0.292 |  Val. f1: 0.49 | Val. precision: 0.49 | Val. recall: 0.52



## 4.- Predecir datos de la competencia

Ahora, a partir de los datos de test y nuestro modelo entrenado, predeciremos las etiquetas que serán evaluadas en la competencia.

In [0]:
def predict_labels(model, iterator, criterion):

    model.eval()
    list_predictions_batch = []
    texts = []

    with torch.no_grad():

        for batch in iterator:
            text = batch.text
            texts.append(torch.transpose(text, 0, 1).tolist())

            # Predecir los tags del batch
            predictions_batch = model(text)
            
            # Hacer las oraciones y no las palabras el primer indice
            predictions_batch = torch.transpose(predictions_batch, 0, 1)
            
            predicted_tags_batch = []
            for predictions_sent in predictions_batch:
                sent_tags = []
                # extraer la clase (el indice de la probabilidad predicha mas alta)
                for prediction_tag in predictions_sent:
                    argmax_index = prediction_tag.topk(1)[1]
                    sent_tags.append(argmax_index)

                predicted_tags_batch.append(sent_tags)
            
            list_predictions_batch.append(predicted_tags_batch)

    return texts, list_predictions_batch

test_texts, test_predictions = predict_labels(model, test_iterator , criterion)


Y transformaremos los vectores de los tags y el texto a sus respectivos tokens partir del vocabulario de los `fields` TEXT y NER_TAGS

In [0]:
def get_tokens_from_vocab(test_texts, field):
  tokens = []
  for batch in test_texts:
      for sent in batch:
          token_batch = []
          for token in sent:
              token_batch.append(field.vocab.itos[token])
          tokens.append(token_batch)
  return tokens

In [0]:
tags = get_tokens_from_vocab(test_predictions, NER_TAGS)
sentences = get_tokens_from_vocab(test_texts, TEXT)

# Ejemplo:
print(tags[0])
print(sentences[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']
['La', '<unk>', ',', '23', 'may', '(', 'EFECOM', ')', '.']


Filtramos \<PAD\>

In [0]:
def filter_pads(sentences,tags):

  filter_sentences = []
  filter_labels = []
  for sent,labels in zip(sentences,tags):
      filter_sentence = []
      filter_label = []
      
      for word,label in zip(sent,labels):
          if word != '<pad>':
              filter_sentence.append(word)
              filter_label.append(label)

      filter_sentences.append(filter_sentence)
      filter_labels.append(filter_label)

  return filter_sentences, filter_labels
    
filter_sentences, filter_labels = filter_pads(sentences,tags)

In [0]:
print("Número de palabras: {}, Número de tags predichos: {}".format(sum([len(sent) for sent in filter_sentences]),sum([len(sent) for sent in filter_labels])))

Número de palabras: 51533, Número de tags predichos: 51533


### Generar el archivo para la submission

In [0]:
import os, shutil

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt','w')
for sent,labels in zip(sentences,filter_labels):
    for word,label in zip(sent,labels):
        f.write(word+' '+label+'\n')
    f.write('\n')
f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')

In [0]:
# A veces no funciona a la primera. Ejecutar mas de una vez para obtener el archivo...
from google.colab import files
files.download('predictions.zip')  